In [1]:
!pip install tensorflow-gpu

     |████████████████████████████████| 377.0MB 57kB/s 


     |████████████████████████████████| 993kB 2.8MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp drive/'My Drive'/'Colab Notebooks'/X.pickle .
!cp drive/'My Drive'/'Colab Notebooks'/y.pickle .

In [6]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time


NAME = 'Cats-vs-Dogs-cnn-64x2-{}'.format(time.time())

tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))

# we separate memory for few sessions
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

# load data from pickle files
X = pickle.load(open('X.pickle', 'rb'))
y = pickle.load(open('y.pickle', 'rb'))

# normalize our data
# 255 is maximum value for pixel data
X = X / 255.0

# initialize our model
model = Sequential()
# add layers to our model
model.add(Conv2D(64, (3, 3), input_shape = X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

model.fit(X, y, batch_size=32, epochs=10, validation_split=0.3, callbacks=[tensorboard])

Train on 17462 samples, validate on 7484 samples
Epoch 1/10
17462/17462 [==============================] - 25s 1ms/sample - loss: 0.6558 - acc: 0.6136 - val_loss: 0.6198 - val_acc: 0.6618
Epoch 2/10
17462/17462 [==============================] - 24s 1ms/sample - loss: 0.5661 - acc: 0.7115 - val_loss: 0.5598 - val_acc: 0.7206
Epoch 3/10
17462/17462 [==============================] - 24s 1ms/sample - loss: 0.4755 - acc: 0.7749 - val_loss: 0.5106 - val_acc: 0.7599
Epoch 4/10
17462/17462 [==============================] - 24s 1ms/sample - loss: 0.3803 - acc: 0.8284 - val_loss: 0.4916 - val_acc: 0.7698
Epoch 5/10
17462/17462 [==============================] - 24s 1ms/sample - loss: 0.2763 - acc: 0.8832 - val_loss: 0.5505 - val_acc: 0.7682
Epoch 6/10
17462/17462 [==============================] - 24s 1ms/sample - loss: 0.1595 - acc: 0.9392 - val_loss: 0.6985 - val_acc: 0.7584
Epoch 7/10
17462/17462 [==============================] - 24s 1ms/sample - loss: 0.0744 - acc: 0.9755 - val_loss: 0.8

In [7]:
# how to run Tensor Board on Google Colab
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

LOG_DIR = './logs'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

get_ipython().system_raw('./ngrok http 6006 &')


--2019-08-14 07:40:47--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 3.217.182.206, 34.231.75.48, 3.220.125.240, ...
Connecting to bin.equinox.io (bin.equinox.io)|3.217.182.206|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13607069 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  12.98M  26.9MB/s    in 0.5s    

2019-08-14 07:40:48 (26.9 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13607069/13607069]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [8]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://a6fff7ed.ngrok.io
